In [23]:
import csv
import numpy as np
import pandas as pd
# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon
from math import factorial
import datetime
import time
import scipy
import os, os.path

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb


import sys
start_time = time.time()

In [24]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')
sys.path.append('/home/hnoorazar/remote_sensing_codes/')

import remote_sensing_core as rc
import remote_sensing_core as rcp


In [25]:
data_base = "/Users/hn/Documents/01_research_data/remote_sensing/01_NDVI_TS/" + \
            "04_Irrigated_eastern_Cloud70/Grant_2018_irrigated/" + \
            "Grant_Irrigated_EVI_2018_NassIn_NotCorrectYears/"

data_base = "/Users/hn/Documents/01_research_data/remote_sensing/" + \
            "01_NDVI_TS/04_Irrigated_eastern_Cloud70/Grant_2018_irrigated/" + \
            "savitzky_EVI/Grant_Irrigated_2018_no_plot/"

param_dir = "/Users/hn/Documents/00_GitHub/Ag/remote_sensing/parameters/"

double_crop_potens = pd.read_csv(param_dir + "double_crop_potential_plants.csv")

In [26]:
# Sav_win_size, sav_order, delt
parameters = [(3, 1, 0.1), (3, 1, 0.2), (3, 1, 0.3), (3, 1, 0.4),
              (5, 1, 0.1), (5, 1, 0.2), (5, 1, 0.3), (5, 1, 0.4),
              (7, 1, 0.1), (7, 1, 0.2), (7, 1, 0.3), (7, 1, 0.4),
              (9, 1, 0.1), (9, 1, 0.2), (9, 1, 0.3), (9, 1, 0.4),

              (3, 2, 0.1), (3, 2, 0.2), (3, 2, 0.3), (3, 2, 0.4),
              (5, 2, 0.1), (5, 2, 0.2), (5, 2, 0.3), (5, 2, 0.4),
              (7, 2, 0.1), (7, 2, 0.2), (7, 2, 0.3), (7, 2, 0.4),
              (9, 2, 0.1), (9, 2, 0.2), (9, 2, 0.3), (9, 2, 0.4),

              # (3, 3, 0.1), (3, 3, 0.2), (3, 3, 0.3), (3, 3, 0.4),
              (5, 3, 0.1), (5, 3, 0.2), (5, 3, 0.3), (5, 3, 0.4),
              (7, 3, 0.1), (7, 3, 0.2), (7, 3, 0.3), (7, 3, 0.4),
              (9, 3, 0.1), (9, 3, 0.2), (9, 3, 0.3), (9, 3, 0.4)]


In [27]:
# form a data table for output.
output_columns = ['params',
                  'NassIn_AllFields_AllYears', 
                  'NassOut_AllFields_AllYears',
                  'NassIn_AllFields_CorrectYear', 
                  'NassOut_AllFields_CorrectYear',
                  
                  'NassIn_DoublePoten_AllYears', 
                  'NassOut_DoublePoten_AllYears',
                  'NassIn_DoublePoten_CorrectYear', 
                  'NassOut_DoublePoten_CorrectYear'
                 ]

output_df = pd.DataFrame(data=None,
                         index=np.arange(len(parameters)), 
                         # index = parameters,
                         columns = output_columns)
output_df['params'] = parameters

In [28]:
output_df.shape

(44, 9)

In [13]:
# for num, param in enumerate(parameters):
#     print("Parameter {}: {}".format(num, param))

In [14]:
for num, param in enumerate(parameters):
    Sav_win_size = param[0]
    sav_order = param[1]
    delt = param[2]
    
    data_dir = data_base + "delta" + str(delt) + \
               "_Sav_win" + str(Sav_win_size) + "_Order" + str(sav_order) + "/"
    curr_table = pd.read_csv(data_dir + "all_poly_and_maxs_savitzky.csv", low_memory=False)
    
    #
    # drop last empty row
    #
    curr_table.drop(curr_table.tail(1).index, inplace=True)
    
    """
    The data table includes all maximum information. 
    So, each field is repeated several times.
    We need to get unique fields.
    """
    curr_table.drop(['max_Doy', 'max_value'], axis=1, inplace=True)
    curr_table.drop_duplicates(inplace=True)
    
    ###
    ###  Pick those with more than two peaks in them
    ###
    curr_table = curr_table[curr_table["max_count"] >= 2]

    curr_table["DataSrc"] = curr_table["DataSrc"].str.lower()
    ##########################################################################################
    ##########################################################################################
    ##########################################################################################
    ##########################################################################################
    ##########################################################################################
    
    ###
    ###  NassIn, AllFields, AllYears 
    ###  i.e. Everything other than those with 
    ###  no maximum detected on their time series.
    ###
    NassIn_AllFields_AllYears_Acr = np.sum(curr_table['ExctAcr'])
    #     print ("1) is NASS in? should be ..." )
    #     print (curr_table.DataSrc.unique())
    #     print("_____________________________________________")

    ###
    ###  NassOut_AllFields_AllYears
    ###
    NassOut_AllFields_AllYears = curr_table[curr_table.DataSrc != 'nass'].copy()

    #     print ("2) is NASS in? should NOT be ...")
    #     print (NassOut_AllFields_AllYears.DataSrc.unique())
    #     print("_____________________________________________")

    NassOut_AllFields_AllYears_Acr = np.sum(NassOut_AllFields_AllYears['ExctAcr'])
    del(NassOut_AllFields_AllYears)

    ###
    ###  NassIn AllFields CorrectYear
    ###
    NassIn_AllFields_CorrectYear = curr_table[curr_table["LstSrvD"].str.contains("2018", na=False)].copy()

    #     print ("3) is NASS in? should be ...")
    #     print ( NassIn_AllFields_CorrectYear.DataSrc.unique())
    #     print("_____________________________________________")

    NassIn_AllFields_CorrectYear_Acr = np.sum(NassIn_AllFields_CorrectYear['ExctAcr'])
    del(NassIn_AllFields_CorrectYear)

    ###
    ###  NassOut AllFields CorrectYear
    ###
    NassOut_AllFields = curr_table[curr_table.DataSrc != 'nass'].copy()
    NassOut_AllFields_CorrectYear = \
                 NassOut_AllFields[NassOut_AllFields["LstSrvD"].str.contains("2018", na=False)].copy()
    
    #     print ("4) is NASS in? should NOT be ...")
    #     print (NassOut_AllFields_CorrectYear.DataSrc.unique())
    #     print("_____________________________________________")

    NassOut_AllFields_CorrectYear_Acr = np.sum(NassOut_AllFields_CorrectYear['ExctAcr'])
    del(NassOut_AllFields, NassOut_AllFields_CorrectYear)

    ###############################################################
    #####
    #####      double potentials
    #####
    ###############################################################

    curr_double_poten = curr_table[curr_table.CropTyp.isin(double_crop_potens['Crop_Type'])]
    del(curr_table)

    ###
    ###  NassIn, double potential, AllYears (i.e. Everything other than non-max)
    ###
    NassIn_DoublePoten_AllYears_Acr = np.sum(curr_double_poten['ExctAcr'])
    
    #     print ("1) is NASS in? should be ...")
    #     print (curr_double_poten.DataSrc.unique())
    #     print("_____________________________________________")

    ###
    ###  NassOut, double potential, AllYears
    ###
    NassOut_DoublePoten_AllYears = curr_double_poten[curr_double_poten.DataSrc != 'nass'].copy()
    NassOut_DoublePoten_AllYears_Acr = np.sum(NassOut_DoublePoten_AllYears['ExctAcr'])

    #     print ("2) is NASS in? should NOT be ...") 
    #     print (NassOut_DoublePoten_AllYears.DataSrc.unique())
    #     print("_____________________________________________")

    ###
    ###  NassIn, double potential, CorrectYear
    ###

    NassIn_DoublePoten_CorrectYear = \
                curr_double_poten[curr_double_poten["LstSrvD"].str.contains("2018", na=False)].copy()
    NassIn_DoublePoten_CorrectYear_Acr = np.sum(NassIn_DoublePoten_CorrectYear['ExctAcr'])
    
    #     print ("3) is NASS in? should be ...")
    #     print (NassIn_DoublePoten_CorrectYear.DataSrc.unique())
    #     print("_____________________________________________")

    del(NassIn_DoublePoten_CorrectYear)

    ###
    ###  NassOut, double potential, CorrectYear
    ###

    NassOut_DoublePoten = curr_double_poten[curr_double_poten.DataSrc != 'nass'].copy()
    NassOut_DoublePoten_CorrectYear = \
                 NassOut_DoublePoten[NassOut_DoublePoten["LstSrvD"].str.contains("2018", na=False)].copy()

    #     print ("4) is NASS in? should NOT be ...")
    #     print (NassOut_DoublePoten_CorrectYear.DataSrc.unique())
    #     print("_____________________________________________")

    NassOut_DoublePoten_CorrectYear_Acr = np.sum(NassOut_DoublePoten_CorrectYear['ExctAcr'])
    del(NassOut_DoublePoten, NassOut_DoublePoten_CorrectYear)
    
    ###############################################################
    #####
    #####      assemble the row and put it in output dataframe
    #####
    ###############################################################
    
    row = [NassIn_AllFields_AllYears_Acr, NassOut_AllFields_AllYears_Acr,
           NassIn_AllFields_CorrectYear_Acr, NassOut_AllFields_CorrectYear_Acr,
           NassIn_DoublePoten_AllYears_Acr, NassOut_DoublePoten_AllYears_Acr,
           NassIn_DoublePoten_CorrectYear_Acr, NassOut_DoublePoten_CorrectYear_Acr]
    output_df.iloc[num, 1: ] = row
    
    

In [15]:
write_path = "/Users/hn/Desktop/"
filename = write_path + "Grant_2018_irrigated_acreages_DoublePeaks.csv"
output_df.to_csv(filename, index = False)